In [1]:
import MDAnalysis as mda
import MDAnalysis.transformations
import nglview as nv

In [2]:
def get_universe(file_path: str) -> tuple:
    """
    get PARM7, DCD files and return universe
    """
    topology = file_path + "somd.prm7"
    trajectory = file_path + "traj000000002.dcd"
    if not os.path.isfile(trajectory):
        trajectory = file_path + "traj000000001.dcd"
    return mda.Universe(topology, trajectory, topology_format="PARM7")

In [3]:
path = f"../outputs/SOMD/lig_h_1~lig_h_2/bound/lambda_0.0000/"
universe = get_universe(path)
workflow = [mda.transformations.unwrap(universe.atoms)]
universe.trajectory.add_transformations(*workflow)
ligand = universe.select_atoms("resname LIG")

In [4]:
carbon_and_dummy = ligand.select_atoms("name C02 C1")

In [5]:
bonds = carbon_and_dummy.bonds
n_frames = universe.trajectory.n_frames
bond_lengths = []
for frame in range(n_frames):
    universe.trajectory[frame]
    bond_lengths = [(atom1.ix, atom2.ix, bond_length) for (atom1, atom2, bond_length) in zip(bonds.atom1, bonds.atom2, bonds.bonds())]


In [11]:
nv.show_mdanalysis(ligand)

NGLWidget(max_frame=499)

In [ ]:
bond_lengths